### DOWNLOAD THE 'CHORD-SCALE PACK (OR dataset)' FROM FREESOUND

In [94]:
import sys,os
sys.path.append(os.path.join(os.path.dirname('__file__'), 'freesound-python/'))

import freesound
client = freesound.FreesoundClient()
client.set_token("1Dio5sf5Kq7uxPPi4WSQljr4EncEzRjuVdlMV8z8","token")
dataDir = 'Chord-Scale_Dataset'

Pack name: Chord-Scale Dataset
Num results: 41
<Sound: id="424916", name="toprak_major2.aiff">
	- toprak_major2.aiff by emirdemirel , with duration of
47.7013 s 
<Sound: id="424915", name="toprak_major.aiff">
	- toprak_major.aiff by emirdemirel , with duration of
11.6245 s 
<Sound: id="424914", name="toprak_lydianb7.aiff">
	- toprak_lydianb7.aiff by emirdemirel , with duration of
59.2536 s 
<Sound: id="424913", name="toprak_lydianb71.aiff">
	- toprak_lydianb71.aiff by emirdemirel , with duration of
95.7651 s 
<Sound: id="424912", name="toprak_lydian1.aiff">
	- toprak_lydian1.aiff by emirdemirel , with duration of
66.9541 s 
<Sound: id="424911", name="toprak_lydian2.aiff">
	- toprak_lydian2.aiff by emirdemirel , with duration of
73.3138 s 
<Sound: id="424910", name="toprak_melmin.aiff">
	- toprak_melmin.aiff by emirdemirel , with duration of
117.707 s 
<Sound: id="424909", name="toprak_altered1.aiff">
	- toprak_altered1.aiff by emirdemirel , with duration of
70.9846 s 
<Sound: id="42490

In [ ]:
pack = client.get_pack(24075)
print("Pack name:", pack.name)
results_pager = pack.get_sounds(
    page_size=41,
    fields="id,name,username,duration,previews"
)

print("Num results:", results_pager.count)


if os.path.exists(dataDir) != 1:
    os.mkdir(dataDir)
os.chdir(dataDir)    
for sound in results_pager:
    print(sound)
    print("\t-", sound.name, "by", sound.username, ", with duration of",)
    print(sound.duration, "s ",)
    sound.retrieve_preview(".",sound.name+".mp3")
    #os.rename(sound.name+".mp3",queryStr+"_"+str(cnt)+".mp3")
    
print(sound.duration, "s ",)

print("Files are copied! check your folder")
os.chdir('..')

### FEATURE EXTRACTION

In [95]:
from utilities import *

params=AnalysisParams(200,100,'hann',2048,44100)


Chord-Scale_Dataset/                     docker-compose.yml
Chord-scale_Detection.ipynb              scales_visualizeHPCP.ipynb
DownloadDataFromFreeSound_noToken.ipynb  utilities.py
